<a href="https://colab.research.google.com/github/keshavkmr48/Projects/blob/master/Copy_of_sequential_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Sequential model

**Author:** [keshav](https://www.linkedin.com/in/keshavkmr076/)<br>
**Date created:** 2020/05/13<br>
**Last modified:** 2020/05/13<br>
**Description:** Complete guide to the Sequential model.

## Setup


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## When to use a Sequential model

A `Sequential` model is appropriate for **a plain stack of layers**
where each layer has **exactly one input tensor and one output tensor**.

Schematically, the following `Sequential` model:


In [0]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)


is equivalent to this function:


In [0]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))


But there are clearly some difference in both of them. One way to find those differences is following - 

In [0]:
print("Number of methods available for model :",len(set(model.__dir__())))
print("Number of methods available for layer3 :",len(set(layer3.__dir__())))
set(model.__dir__())-set(layer3.__dir__())

Number of methods available for model : 274
Number of methods available for layer3 : 194


['_in_multi_worker_mode',
 'predict',
 'layers',
 '_layer_call_argspecs',
 '_is_graph_network',
 '_assert_weights_created',
 'make_train_function',
 'output_names',
 '_check_call_args',
 'stop_training',
 'predict_function',
 '_get_callback_model',
 '_validate_compile',
 'metrics_names',
 '_run_internal_graph',
 'add',
 'test_function',
 '_reset_compile_cache',
 '_undeduplicated_weights',
 '_graph_network_add_metric',
 '_assert_compile_was_called',
 '_is_compiled',
 'input_names',
 'predict_proba',
 'evaluate',
 '_insert_layers',
 '_scope',
 '_graph',
 'compiled_metrics',
 'to_yaml',
 '_get_save_spec',
 'get_layer',
 '_get_optimizer',
 'predict_on_batch',
 'reset_states',
 'reset_metrics',
 '_compile_was_called',
 '_training_state',
 '_should_eval',
 '_trackable_saver',
 'predict_classes',
 'train_on_batch',
 '_layer_checkpoint_dependencies',
 'make_test_function',
 '_compiled_trainable_state',
 '_init_subclassed_network',
 '_set_save_spec',
 'evaluate_generator',
 '_handle_deferred_la

A Sequential model is **not appropriate** when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch
model)


## Creating a Sequential model

You can create a Sequential model by passing a list of layers to the Sequential
constructor:


In [0]:
model1 = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)


Its layers are accessible via the `layers` attribute:


In [0]:
model1.layers


You can also create a Sequential model incrementally via the `add()` method:


In [0]:
model2 = keras.Sequential()
model2.add(layers.Dense(2, activation="relu"))
model2.add(layers.Dense(3, activation="relu"))
model2.add(layers.Dense(4))


Note that there's also a corresponding `pop()` method to remove layers:
a Sequential model behaves very much like a list of layers.


In [0]:
model1.pop()
print(len(model1.layers))  # 2


2


In [0]:
model2.pop()
print(len(model2.layers))  # 2

2


Also note that the Sequential constructor accepts a `name` argument, just like
any layer or model in Keras. This is useful to annotate TensorBoard graphs
with semantically meaningful names.


In [0]:
model3 = keras.Sequential(name="my_sequential")
model3.add(layers.Dense(2, activation="relu", name="layer1"))
model3.add(layers.Dense(3, activation="relu", name="layer2"))
model3.add(layers.Dense(4, name="layer3"))


In [0]:
model3.layers

## Specifying the input shape in advance

Generally, all layers in Keras need to know the shape of their inputs
in order to be able to create their weights. So when you create a layer like
this, initially, it has no weights:


In [0]:
layer = layers.Dense(3)
layer.weights  # Empty


[]

It creates its weights the first time it is called on an input, since the shape
of the weights depends on the shape of the inputs:


In [0]:
# Call layer on a test input
x1 = tf.ones((1, 4))
y1 = layer(x1)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)


[<tf.Variable 'dense_6/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.25667357,  0.11591733, -0.72061217],
        [-0.5539725 , -0.84571224,  0.8368032 ],
        [ 0.6450931 ,  0.52548885,  0.7324097 ],
        [ 0.16038036,  0.82514477,  0.7748076 ]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

Naturally, this also applies to Sequential models. When you instantiate a
Sequential model without an input shape, it isn't "built": it has no weights
(and calling
`model.weights` results in an error stating just this). The weights are created
when the model first sees some input data:


In [0]:
model.weights

[<tf.Variable 'sequential/layer1/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.14032853, -0.5587893 ],
        [ 0.6985451 , -0.33764166],
        [ 0.3068298 , -0.8465407 ]], dtype=float32)>,
 <tf.Variable 'sequential/layer1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'sequential/layer2/kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[-0.50713867,  0.18243194,  0.24092853],
        [-0.67550975, -0.9485998 ,  0.06401491]], dtype=float32)>,
 <tf.Variable 'sequential/layer2/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'sequential/layer3/kernel:0' shape=(3, 4) dtype=float32, numpy=
 array([[-0.10425252, -0.1736409 ,  0.8275808 , -0.86577594],
        [ 0.32070327,  0.26072788,  0.6076361 , -0.02242821],
        [ 0.57258534, -0.10138386,  0.46188855,  0.7239015 ]],
       dtype=float32)>,
 <tf.Variable 'sequential/layer3/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], 

In [0]:
model1.weights

ValueError: ignored

In [0]:
model2.weights

ValueError: ignored

In [0]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6


Number of weights after calling the model: 6


Once a model is "built", you can call its `summary()` method to display its
contents:


In [0]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              multiple                  10        
_________________________________________________________________
dense_8 (Dense)              multiple                  9         
_________________________________________________________________
dense_9 (Dense)              multiple                  16        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


However, it can be very useful when building a Sequential model incrementally
to be able to display the summary of the model so far, including the current
output shape. In this case, you should start your model by passing an `Input`
object to your model, so that it knows its input shape from the start:


In [0]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


Note that the `Input` object is not displayed as part of `model.layers`, since
it isn't a layer:


In [0]:
model.layers


A simple alternative is to just pass an `input_shape` argument to your first
layer:


In [0]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


Models built with a predefined input shape like this always have weights (even
before seeing any data) and always have a defined output shape.

In general, it's a recommended best practice to always specify the input shape
of a Sequential model in advance if you know what it is.


## A common debugging workflow: `add()` + `summary()`

When building a new Sequential architecture, it's useful to incrementally stack
layers with `add()` and frequently print model summaries. For instance, this
enables you to monitor how a stack of `Conv2D` and `MaxPooling2D` layers is
downsampling image feature maps:


In [0]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
____________________________

Very practical, right?



## What to do once you have a model

Once your model architecture is ready, you will want to:

- Train your model, evaluate it, and run inference. See our
[guide to training & evaluation with the built-in loops](
    /guides/training_with_built_in_methods/)
- Save your model to disk and restore it. See our
[guide to serialization & saving](/guides/serialization_and_saving/).
- Speed up model training by leveraging multiple GPUs. See our
[guide to multi-GPU and distributed training](distributed_training).


## Feature extraction with a Sequential model

Once a Sequential model has been built, it behaves like a [Functional API
model](/guides/functional_api/). This means that every layer has an `input`
and `output` attribute. These attributes can be used to do neat things, like
quickly
creating a model that extracts the outputs of all intermediate layers in a
Sequential model:


In [0]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)

In [0]:
initial_model.inputs

[<tf.Tensor 'input_3:0' shape=(None, 250, 250, 3) dtype=float32>]

In [0]:
[layer.output for layer in initial_model.layers]

[<tf.Tensor 'conv2d_6/Identity:0' shape=(None, 123, 123, 32) dtype=float32>,
 <tf.Tensor 'conv2d_7/Identity:0' shape=(None, 121, 121, 32) dtype=float32>,
 <tf.Tensor 'conv2d_8/Identity:0' shape=(None, 119, 119, 32) dtype=float32>]

In [0]:
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)
feature_extractor.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 119, 119, 32)      9248      
Total params: 20,928
Trainable params: 20,928
Non-trainable params: 0
_________________________________________________________________


In [0]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)


Here's a similar example that only extract features from one layer:


In [0]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
feature_extractor.summary()
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 123, 123, 32)      2432      
_________________________________________________________________
my_intermediate_layer (Conv2 (None, 121, 121, 32)      9248      
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________










## Transfer learning with a Sequential model

Transfer learning consists of freezing the bottom layers in a model and only training
the top layers. If you aren't familiar with it, make sure to read our [guide
to transfer learning](/guides/transfer_learning/).

Here are two common transfer learning blueprint involving Sequential models.

First, let's say that you have a Sequential model, and you want to freeze all
layers except the last one. In this case, you would simply iterate over
`model.layers` and set `layer.trainable = False` on each layer, except the
last one. Like this:

```python
model = keras.Sequential([
    keras.Input(shape=(784))
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
model.compile(...)
model.fit(...)
```

Another common blueprint is to use a Sequential model to stack a pre-trained
model and some freshly initialized classification layers. Like this:

```python
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

# Compile & train
model.compile(...)
model.fit(...)
```

If you do transfer learning, you will probably find yourself frequently using
these two patterns.


That's about all you need to know about Sequential models!

To find out more about building models in Keras, see:

- [Guide to the Functional API](/guides/functional_api/)
- [Guide to making new Layers & Models via subclassing](
    /guides/making_new_layers_and_models_via_subclassing/)
